In [1]:
filename ="./FGF4_datasets/ncbi_dataset/data/gene.fna"

In [2]:
from Bio import SeqIO

In [5]:
seq_object= SeqIO.read(filename,"fasta")


In [6]:
print(seq_object.description)

NC_037356.1:c47003919-47001981 FGF4 [organism=Bos taurus] [GeneID=618474] [chromosome=29]


In [7]:
sequence= seq_object.seq
print(len(sequence))

1939


In [8]:
import random
def truncate_sequence(sequence):
    if len(sequence) > 400:
        # Calculate number of deletions required
        num_deletions = len(sequence) - 400
        # Randomly select positions to delete
        delete_positions = random.sample(range(len(sequence)), num_deletions)
        # Create truncated sequence
        truncated_sequence = ''.join([sequence[i] for i in range(len(sequence)) if i not in delete_positions])
        return truncated_sequence
    else:
        return sequence

# Function to introduce mutations based on specified rules
def introduce_mutations(sequence, k):
    # Insertions
    for _ in range(k):
        pos = random.randint(0, len(sequence))
        sequence = sequence[:pos] + random.choice('ACGT') + sequence[pos:]
    
    # Deletions
    for _ in range(k):
        pos = random.randint(0, len(sequence) - 1)
        sequence = sequence[:pos] + sequence[pos+1:]
    
    # Single-point mutations
    for _ in range(k):
        pos = random.randint(0, len(sequence) - 1)
        sequence = sequence[:pos] + random.choice('ACGT'.replace(sequence[pos], '')) + sequence[pos+1:]
    
    # Generate X1 variant
    start1 = 2 * k
    repeat_seq1 = sequence[start1:start1 + k]  # Subsequence starting at 2k
    start2 = 400 - 2 * k
    delete_seq1 = sequence[start2:start2 + k]  # Subsequence starting at 400-2k

    X1 = sequence[:start2] + repeat_seq1 + sequence[start2:]  # Insert repeat_seq1
    X1 = X1.replace(delete_seq1, '')  # Delete delete_seq1 from X1

    # Generate X2 variant
    start3 = 400 - 2 * k
    repeat_seq2 = sequence[start3:start3 + k]  # Subsequence starting at 400-2k
    start4 = 2 * k
    delete_seq2 = sequence[start4:start4 + k]  # Subsequence starting at 2k

    X2 = sequence[:start4] + repeat_seq2 + sequence[start4:]  # Insert repeat_seq2
    X2 = X2.replace(delete_seq2, '')  # Delete delete_seq2 from X2

    return X1, X2




In [9]:
truncated_sequence = truncate_sequence(sequence)
print(len(truncated_sequence))


400


In [10]:
print(truncated_sequence)

AGCGGCGCTGGTTGAGGCCCCGCAGGGCGGGGGAGGCCGCCTTGGTTGGCTACGGTGCGGCAGTAGTGTGGCGCCTCCATGTCAGGCGCGCAGTCCGCTACCCGCGAGATCCTTTCGGCGAACCGGGCGAGCAGACCGGCGCCGGCGGACACTCCCCAAGGGGGGGAGCGGTCTCTGAGCGGTGTGCGGGTCCCCGGTAGGCCCGCAGGCCTGGGACGCTCCGTGACTCGGCCTTACGTCTGCCCGCCTTCAACGCGGTCCCCAATGACCACGATCTGGATCAGCCGCCAAGGGGCGCTTAAGTATGTGGGAAACGGGGCGCTTGCTGCTGCTCCGTCGAGCTCAGGAGAGACCGCCAGGTCCTCGGCAACCACCTCCACCCACAAACGCAGAGTCTGGA


In [11]:
k = 10
X1, X2 = introduce_mutations(truncated_sequence, k)

print(truncated_sequence)
print("\nVariant X1:")
print(X1)
print("\nVariant X2:")
print(X2)


AGCGGCGCTGGTTGAGGCCCCGCAGGGCGGGGGAGGCCGCCTTGGTTGGCTACGGTGCGGCAGTAGTGTGGCGCCTCCATGTCAGGCGCGCAGTCCGCTACCCGCGAGATCCTTTCGGCGAACCGGGCGAGCAGACCGGCGCCGGCGGACACTCCCCAAGGGGGGGAGCGGTCTCTGAGCGGTGTGCGGGTCCCCGGTAGGCCCGCAGGCCTGGGACGCTCCGTGACTCGGCCTTACGTCTGCCCGCCTTCAACGCGGTCCCCAATGACCACGATCTGGATCAGCCGCCAAGGGGCGCTTAAGTATGTGGGAAACGGGGCGCTTGCTGCTGCTCCGTCGAGCTCAGGAGAGACCGCCAGGTCCTCGGCAACCACCTCCACCCACAAACGCAGAGTCTGGA

Variant X1:
GCGGCGCTGGTTGAGGCCCCGCAGGGCGGGGGAGGCCGCCTTGGTAGGCTACGGTGCGGCAGTCGTGTGGCGCCTCCATGTCAGGCGTCGCAGTCCGCTACCAGCGAGATCCTTTCGGGCGAACCGGGGCGAGAGACCGGCGCCGGCGGACACTCCCAAGGGGGGAGCGGTCTCTGAGCGGTTTGCGTGACCCCGGTTAGGCTCCGCCGGCCTGGGACGCTCCGTGACTCGGCCTTACGTCTGCCCGCCTTCAAAGCGTCCCAATGACCACGATCTGGATCAGACCGCCAAGGGGCGCTTAAGTATGTGGGAAACGGGGCGCTTGCTGCTGCTCCTCGAGCTCAGGAGTGACCCCCAGTCCTCAGGCAACCACCTCCACCGCAGGGCGGGAGAGTCTGGA

Variant X2:
GCGGCGCTGGTTGAGGCCCCCACTAAACGCGGAGGCCGCCTTGGTAGGCTACGGTGCGGCAGTCGTGTGGCGCCTCCATGTCAGGCGTCGCAGTCCGCTACCAGCGAGATCCTTTCGGGCGAACCGGGGCGAGAGACCGGCGCCGGCGGACACTCCCAAGGGGGGAGCGGTC

In [12]:
from Bio import pairwise2
from Bio.Seq import Seq
from Bio import SeqIO

# Function to perform Needleman-Wunsch alignment
def needleman_wunsch_alignment(seq1, seq2):
    alignments = pairwise2.align.globalms(seq1, seq2, 1, -1, -1, -1)
    best_alignment = alignments[0]
    return best_alignment

# Function to perform FASTA alignment
def fasta_alignment(seq1, seq2):
    alignments = pairwise2.align.localms(seq1, seq2, 1, -1, -1, -1)
    best_alignment = alignments[0]
    return best_alignment


/Users/vidyaanildhulappanavar/miniconda3/lib/python3.10/site-packages/Bio/pairwise2.py:278: BiopythonDeprecationWarning: Bio.pairwise2 has been deprecated, and we intend to remove it in a future release of Biopython. As an alternative, please consider using Bio.Align.PairwiseAligner as a replacement, and contact the Biopython developers if you still need the Bio.pairwise2 module.
  warnings.warn(


In [13]:
alignment_nw = needleman_wunsch_alignment(X1, X2)
alignment_fasta = fasta_alignment(X1, X2)

print("\nNeedleman-Wunsch Alignment:")
print(pairwise2.format_alignment(*alignment_nw))
print("\nFASTA Alignment:")
print(pairwise2.format_alignment(*alignment_fasta))




Needleman-Wunsch Alignment:
GCGGCGCTGGTTGAGGCCCCGCAGGG--CGGGGGAGGCCGCCTTGGTAGGCTACGGTGCGGCAGTCGTGTGGCGCCTCCATGTCAGGCGTCGCAGTCCGCTACCAGCGAGATCCTTTCGGGCGAACCGGGGCGAGAGACCGGCGCCGGCGGACACTCCCAAGGGGGGAGCGGTCTCTGAGCGGTTTGCGTGACCCCGGTTAGGCTCCGCCGGCCTGGGACGCTCCGTGACTCGGCCTTACGTCTGCCCGCCTTCAAAGCGTCCCAATGACCACGATCTGGATCAGACCGCCAAGGGGCGCTTAAGTATGTGGGAAACGGGGCGCTTGCTGCTGCTCCTCGAGCTCAGGAGTGACCCCCAGTCCTCAGGCAACCACCTCCACCGCAGGG--CGGGAGAGTCTGGA
|||||||||||||||||||| ||...  | |.|||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||| ||...  | |.||||||||||
GCGGCGCTGGTTGAGGCCCC-CACTAAAC-GCGGAGGCCGCCTTGGTAGGCTACGGTGCGGCAGTCGTGTGGCGCCTCCATGTCAGGCGTCGCAGTCCGCTACCAGCGAGATCCTTTCGGGCGAACCGGGGCGAGAGACCGGCGCCGGCGGACACTCCCAA

In [20]:
import time
from Bio import pairwise2
from Bio.Blast import NCBIWWW, NCBIXML


# Function to calculate identity
def calculate_identity_similarity(alignment):
    aligned_seq1, aligned_seq2, score, begin, end = alignment
    alignment_length = len(aligned_seq1)
    num_identical = sum(1 for i in range(alignment_length) if aligned_seq1[i] == aligned_seq2[i] and aligned_seq1[i] != '-')
    identity = num_identical / alignment_length * 100
    return identity

# List of k values to iterate over
k_values = [20, 30, 40, 50]

# Storage for results
results = []

# Iterate over each k value
for k in k_values:
    print(f"\n### Running for k = {k} ###")
    X1, X2 = introduce_mutations(truncated_sequence, k)


    # Measure time and quality for Needleman-Wunsch alignment
    start_time = time.time()
    alignment_nw = needleman_wunsch_alignment(X1, X2)
    nw_time = time.time() - start_time
    nw_score = alignment_nw[2]
    nw_identity = calculate_identity_similarity(alignment_nw)

    # Measure time and quality for FASTA alignment
    start_time = time.time()
    alignment_fasta = fasta_alignment(X1, X2)
    fasta_time = time.time() - start_time
    fasta_score = alignment_fasta[2]
    fasta_identity = calculate_identity_similarity(alignment_fasta)
    
    start_time_blast = time.time()
    result_handle = NCBIWWW.qblast("blastn", "nt", X1)
    blast_record = NCBIXML.read(result_handle)
    blast_score = max(hsp.score for alignment in blast_record.alignments for hsp in alignment.hsps)
    duration_blast = time.time() - start_time_blast

    # Store results for current k value
    results.append({
        'k': k,
        'NW_time': nw_time,
        'NW_score': nw_score,
        'NW_identity': nw_identity,
        'FASTA_time': fasta_time,
        'FASTA_score': fasta_score,
        'FASTA_identity': fasta_identity,
        'Blast_time': duration_blast,
        'BLAST score': blast_score
    })

    # Print results for current k value
    print(f"\nNeedleman-Wunsch Alignment (time: {nw_time:.4f} seconds, score: {nw_score}, identity: {nw_identity:.2f}%):")
    print(pairwise2.format_alignment(*alignment_nw))
    print(f"\nFASTA Alignment (time: {fasta_time:.4f} seconds, score: {fasta_score}, identity: {fasta_identity:.2f}%):")
    print(pairwise2.format_alignment(*alignment_fasta))

# Print summary of results side by side
print("\n### Summary of Results ###")
print(f"{'k':<3} | {'NW Time':<8} | {'NW Score':<8} | {'NW Identity':<12} || {'FASTA Time':<10} | {'FASTA Score':<11} | {'FASTA Identity':<14}")
print("-" * 90)
for result in results:
    print(f"{result['k']:<3} | {result['NW_time']:<8.4f} | {result['NW_score']:<8} | {result['NW_identity']:<12.2f} || {result['FASTA_time']:<10.4f} | {result['FASTA_score']:<11} | {result['FASTA_identity']:<14.2f}")



### Running for k = 20 ###

Needleman-Wunsch Alignment (time: 0.0546 seconds, score: 356.0, identity: 94.06%):
ACCGGCTTGCTTGAAGCCCCGCAGGGCGGGGAAGGCCGCTTGTTTAGGCTACGTGCCGGCA--TTGGTGTCGGCGCCTCTTGTCAGGGCGCGCAGTCCCGCTACCCGCGAGATCCTGTCTGCGAACCGGGCGAGCAATCGCCTACCGCGGACACTCCCCAAGGGGGGGAGCGGTCTCTGAGCGCGTGTGCGGGTCCCCCGAGTAGGCCCGCAGGCCTGGCACGCTCCGTGACTCGCCTACGTCTGCCGCCTTCTACGCGGTCCCCAATGACCACGATCTGGAACAGCCGCCAAGGGCGCTTAACTATGTGGGAAAGCGGGGAGCTGCTGCTGCACTGTCGAGCTCAGAGAGACCGCCAGGTCTGTTTAGGCTACGTGCCGGC-C-AAACGAAACGCAGAGTGGA
||||||||||||||||||||||||||||||||||||||| || |..|||.||...||..||  ||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||| | |..|||.||...||..| | |||||||||||||||||||
ACCGGCTTGCTTGAAGCCCCGCAGGGCGGGGAAGGCCGC-TG-TCGGGCAACCAACCTCCACATTGGTGTCGGCGCCT

In [21]:
print("\n### Summary of Results ###")
print(f"{'k':<3} | {'NW Time':<8} | {'NW Score':<8} | {'NW Identity':<12} || {'Blast_time':<11} | {'BLAST score':<14}")
print("-" * 90)

for result in results:
    print(f"{result['k']:<3} | {result['NW_time']:<8.4f} | {result['NW_score']:<8} | {result['NW_identity']:<12.2f} | {result['Blast_time']:<11} | {result['BLAST score']:<14.2f}")



### Summary of Results ###
k   | NW Time  | NW Score | NW Identity  || Blast_time  | BLAST score   
------------------------------------------------------------------------------------------
20  | 0.0546   | 356.0    | 94.06        | 61.648799896240234 | 56.00         
30  | 0.0988   | 354.0    | 93.17        | 139.4913191795349 | 53.00         
40  | 0.1022   | 320.0    | 88.83        | 79.84892511367798 | 55.00         
50  | 0.1213   | 308.0    | 87.38        | 139.5076401233673 | 51.00         
